<a href="https://colab.research.google.com/github/Maheenms/GoogleCoLab/blob/main/cloud_etl_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
#spark_version = 'spark-<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load file

url = "/content/drugsComTrain_raw.tsv" #enter correct address here

#spark.sparkContext.addFile(url) - No need to use sparkcontext because we are reading it from a local folder

df = spark.read.csv(url, header=True, inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")

df.show(10)

+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|                 _c0|            drugName|           condition|              review|rating|             date|usefulCount|
+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|              206461|           Valsartan|Left Ventricular ...|"""It has no side...|   9.0|     May 20, 2012|         27|
|               95260|          Guanfacine|                ADHD|"""My son is half...|  null|             null|       null|
|We have tried man...|                 8.0|      April 27, 2010|                 192|  null|             null|       null|
|               92703|              Lybrel|       Birth Control|"""I used to take...|  null|             null|       null|
|The positive side...|                 5.0|   December 14, 2009|                  17|  null|             null|       null|
|              1

## Transform DataFrame to fit a schema with data types "text" and "integer"

In [4]:
shop_df = df.select(["drugName","rating"])
shop_df.show()

+--------------------+------+
|            drugName|rating|
+--------------------+------+
|           Valsartan|   9.0|
|          Guanfacine|  null|
|                 8.0|  null|
|              Lybrel|  null|
|                 5.0|  null|
|          Ortho Evra|   8.0|
|Buprenorphine / n...|   9.0|
|              Cialis|   2.0|
|      Levonorgestrel|   1.0|
|        Aripiprazole|  10.0|
|              Keppra|   1.0|
|Ethinyl estradiol...|   8.0|
|          Topiramate|   9.0|
|      L-methylfolate|  null|
|                null|  null|
|                10.0|  null|
|             Pentasa|   4.0|
|    Dextromethorphan|   4.0|
|           Nexplanon|  null|
|                 3.0|  null|
+--------------------+------+
only showing top 20 rows



In [5]:
drug_ratings_df = shop_df.groupby("drugName").agg({"rating": "avg", "drugName":"count"})
drug_ratings_df.show()

+--------------------+-----------------+---------------+
|            drugName|      avg(rating)|count(drugName)|
+--------------------+-----------------+---------------+
|              Dulera|5.285714285714286|             24|
|              Prozac|7.482843137254902|            472|
|               Arava|6.357142857142857|             14|
|              Maxalt|         8.890625|             70|
|          Tazarotene|7.416666666666667|             14|
|           Primidone|6.193548387096774|             41|
|Acetaminophen / d...|6.565217391304348|             23|
|            Treximet|              9.0|             58|
|              Avapro|4.956521739130435|             23|
|             Prograf|8.411764705882353|             18|
|           Auranofin|             10.0|              2|
|    Spiriva Respimat|              7.0|              4|
|   Parafon Forte DSC|              6.0|              2|
|                 1.0|             null|           2278|
|            Dupixent|7.8333333

In [6]:
from pyspark.sql.functions import desc
drug_ratings_df = drug_ratings_df.withColumnRenamed("count(drugName)", "total_ratings")\
                                     .withColumnRenamed("avg(rating)", "avg_rating")
drug_ratings_df.orderBy(desc("avg_rating")).show()

+--------------------+----------+-------------+
|            drugName|avg_rating|total_ratings|
+--------------------+----------+-------------+
|Do not stop the m...|     148.0|            1|
|                   "|      62.0|            1|
|         Ecallantide|      10.0|            1|
|       Sertaconazole|      10.0|            1|
|         Ferumoxytol|      10.0|            2|
|          Ovace Plus|      10.0|            1|
|         Pilocarpine|      10.0|            1|
|             Plendil|      10.0|            2|
|             Privine|      10.0|            9|
|             R-Tanna|      10.0|            2|
|          Filgrastim|      10.0|            2|
|Flanax Pain Reliever|      10.0|            1|
|Salicylic acid / ...|      10.0|            1|
|          Capmist DM|      10.0|            1|
|          Phosphasal|      10.0|            1|
|Chlorpheniramine ...|      10.0|            6|
|           ProCentra|      10.0|            1|
|         Royal jelly|      10.0|       

## Transform DataFrame to fit a schema with data types of "date" and "text"

In [7]:
review_df = df.select(["review", "date"])

In [8]:
review_df.show()

+--------------------+-----------------+
|              review|             date|
+--------------------+-----------------+
|"""It has no side...|     May 20, 2012|
|"""My son is half...|             null|
|                 192|             null|
|"""I used to take...|             null|
|                  17|             null|
|"""This is my fir...| November 3, 2015|
|"""Suboxone has c...|November 27, 2016|
|"""2nd day on 5mg...|November 28, 2015|
|"""He pulled out,...|    March 7, 2017|
|"""Abilify change...|   March 14, 2015|
|""" I Ve had  not...|   August 9, 2016|
|"""I had been on ...| December 8, 2016|
|"""I have been on...|  January 1, 2015|
|"""I have taken a...|             null|
|                null|             null|
|                  54|             null|
|"""I had Crohn&#0...|     July 6, 2013|
|"""Have a little ...|September 7, 2017|
|"""Started Nexpla...|             null|
|                  10|             null|
+--------------------+-----------------+
only showing top

In [9]:
#Drop columns with NA
review_df = review_df.dropna()

In [10]:
date_df = review_df.groupBy('date').agg({"date": "count"})
date_df = date_df.withColumnRenamed("count(date)", "review_count")
date_df.show()

+-----------------+------------+
|             date|review_count|
+-----------------+------------+
|      May 7, 2011|          17|
|    March 2, 2017|          79|
|   March 21, 2013|          25|
|   April 29, 2016|         111|
|     May 30, 2017|          68|
|February 10, 2013|          30|
| October 31, 2015|          79|
| October 27, 2015|         108|
|   April 18, 2015|          43|
|    March 4, 2010|          10|
|   March 25, 2016|          53|
| November 4, 2010|          20|
| January 17, 2009|          12|
|  August 15, 2012|          33|
|February 21, 2009|          21|
|   April 26, 2008|          13|
|September 8, 2010|          19|
| October 29, 2012|          13|
|December 25, 2013|          11|
|      May 5, 2014|          27|
+-----------------+------------+
only showing top 20 rows



In [11]:
date_df.orderBy(desc("review_count")).show()

+------------------+------------+
|              date|review_count|
+------------------+------------+
|     March 1, 2016|         128|
|September 23, 2015|         125|
|  January 27, 2016|         120|
|  February 5, 2016|         120|
| December 15, 2015|         120|
|  January 13, 2016|         118|
|    August 6, 2015|         118|
|  January 20, 2016|         117|
|    April 18, 2017|         117|
|  February 3, 2016|         115|
|   January 5, 2017|         114|
|    March 31, 2016|         114|
|   January 7, 2017|         114|
|     July 12, 2016|         114|
|    August 3, 2016|         113|
|  January 16, 2017|         111|
|    April 29, 2016|         111|
|    April 25, 2017|         111|
| February 26, 2016|         111|
| February 16, 2016|         109|
+------------------+------------+
only showing top 20 rows

